In [29]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [32]:
house_dict = []

for page in range(16, 20):
    # Filter Out Coming Soon, Auction, Home Type: Multi-family, Days On Zillow: 90 Days
    linked = 'https://www.zillow.com/ca/duplex/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-130.93471906436687,"east":-107.64370343936687,"south":31.980790552461904,"north":48.0418909706512},"mapZoom":5,"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"doz":{"value":"90"},"isSingleFamily":{"value":false},"isCondo":{"value":false}},"isListVisible":true}'
    user_agent = 'Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/20100101 Firefox/7.0.1'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    print(soup)

    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            # Identify and return house data
            listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
            full_address = result.find('address', class_="list-card-addr").text
#             state = result.find('h3', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")
            
            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
                
            try:
                zestimate = loop_soup.findAll('p', class_="Text-aiai24-0 sc-eHgmQL sc-kasBVs iZUTqI")[0].text
                if (zestimate == "Zestimate®"):
                    zestimate = 'null'
#                 print('zestimate', zestimate)
                rent_zestimate = loop_soup.findAll('p', class_="Text-aiai24-0 sc-eHgmQL sc-kasBVs iZUTqI")[1].text
            except:
                zestimate = 'null'
#                 print('zestimate', zestimate)
                rent_zestimate = 'null'
#             print(zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(2)
            
        except AttributeError as e:
            continue

print(house_dict)

<html><head><meta content="noindex, nofollow" name="robots"/><link href="https://www.zillowstatic.com/vstatic/80d5e73/static/css/z-pages/captcha.css" media="screen" rel="stylesheet" type="text/css"/><script>
        window._pxAppId = 'PXHYx10rg3';
        window._pxJsClientSrc = '/HYx10rg3/init.js';
        window._pxHostUrl = '/HYx10rg3/xhr';
        window._pxFirstPartyEnabled = true;
        window._pxreCaptchaTheme='light';
    </script><script src="https://captcha.px-cdn.net/PXHYx10rg3/captcha.js?a=c&amp;m=0" type="text/javascript"></script>
<script>
        function getQueryString(name, url) {
            if (!url) url = window.location.href;
            name = name.replace(/[\[\]]/g, "\\$&");
            var regex = new RegExp("[?&]" + name + "(=([^&#]*)|&|#|$)"),
                results = regex.exec(url);
            if (!results) return null;
            if (!results[2]) return '';
            return decodeURIComponent(results[2].replace(/\+/g, " "));
        }
        documen

<html><head><meta content="noindex, nofollow" name="robots"/><link href="https://www.zillowstatic.com/vstatic/80d5e73/static/css/z-pages/captcha.css" media="screen" rel="stylesheet" type="text/css"/><script>
        window._pxAppId = 'PXHYx10rg3';
        window._pxJsClientSrc = '/HYx10rg3/init.js';
        window._pxHostUrl = '/HYx10rg3/xhr';
        window._pxFirstPartyEnabled = true;
        window._pxreCaptchaTheme='light';
    </script><script src="https://captcha.px-cdn.net/PXHYx10rg3/captcha.js?a=c&amp;m=0" type="text/javascript"></script>
<script>
        function getQueryString(name, url) {
            if (!url) url = window.location.href;
            name = name.replace(/[\[\]]/g, "\\$&");
            var regex = new RegExp("[?&]" + name + "(=([^&#]*)|&|#|$)"),
                results = regex.exec(url);
            if (!results) return null;
            if (!results[2]) return '';
            return decodeURIComponent(results[2].replace(/\+/g, " "));
        }
        documen

In [19]:
house_data = pd.DataFrame(house_dict)
house_data

,Bath,Bed,City,Full_address,HOA,House_type,House_website,Listing_type,Lot,Lot_conversion_to_sqft,Parking,Price,Rent_zestimate,Square_Feet,State,Street,Year_build,Zestimate,Zillow_price/sqft,Zip_code
0,2,4,LosAngeles,"1573 N Herbert Ave, Los Angeles, CA 90063",null,Multi Family,https://www.zillow.com/homedetails/1573-N-Herb...,Apartment for sale,null,null,12 spaces,"$775,000","$2,500/mo","1,612",CA,1573 N Herbert Ave,1995,"$775,006",$481,90063
1,3,4,FortBragg,"32200 Simpson Ln, Fort Bragg, CA 95437",null,Multi Family,https://www.zillow.com/homedetails/32200-Simps...,Apartment for sale,null,null,3 spaces,"$925,000","$1,600/mo","2,623",CA,32200 Simpson Ln,1985,"$873,565",$353,95437
2,2,4,Fresno,"2915 N Pleasant Ave, Fresno, CA 93705",null,Multi Family,https://www.zillow.com/homedetails/2915-N-Plea...,Apartment for sale,null,null,2 spaces,"$235,000",$975/mo,"2,108",CA,2915 N Pleasant Ave,1967,"$235,002",$111,93705
3,2,2,YuccaValley,"7226 Grand Ave #A-B, Yucca Valley, CA 92284",null,Multi Family,https://www.zillow.com/homedetails/7226-Grand-...,Apartment for sale,null,null,2 spaces,"$189,900","$1,195/mo","1,240",CA,7226 Grand Ave #A-B,1958,"$189,710",$153,92284
4,4,6,LosAngeles,"8337 W 4th St, Los Angeles, CA 90048",null,Multi Family,https://www.zillow.com/homedetails/8337-W-4th-...,Apartment for sale,null,null,4 spaces,"$1,995,000","$3,350/mo","3,934",CA,8337 W 4th St,1929,"$1,971,111",$507,90048
5,2,2,SanDiego,"1130-32 Sassafras St, San Diego, CA 92103",$395/month,Multi Family,https://www.zillow.com/homedetails/1130-32-Sas...,Apartment for sale,null,null,No Data,"$899,000","$2,400/mo","1,012",CA,1130-32 Sassafras St,1937,"$889,486",$888,92103
6,4,4,LosAngeles,"4656 Maplewood Ave, Los Angeles, CA 90004",None,Multi Family,https://www.zillow.com/homedetails/4656-Maplew...,Apartment for sale,null,null,6 spaces,"$1,195,000","$3,495/mo","2,352",CA,4656 Maplewood Ave,1922,"$1,207,485",$508,90004
7,4,8,SanJose,"791 Farm Dr, San Jose, CA 95136",null,Multi Family,https://www.zillow.com/homedetails/791-Farm-Dr...,For sale by owner,null,null,No Data,"$1,950,000","$3,000/mo","3,490",CA,791 Farm Dr,1963,"$1,654,898",$559,95136
8,3,5,Monrovia,"240 W Cypress Ave, Monrovia, CA 91016",null,Multi Family,https://www.zillow.com/homedetails/240-W-Cypre...,Apartment for sale,null,null,4 spaces,"$1,299,800","$2,700/mo","2,717",CA,240 W Cypress Ave,1947,"$1,294,530",$478,91016
9,4,8,RedBluff,"760 Ash St, Red Bluff, CA 96080",null,Multi Family,https://www.zillow.com/homedetails/760-Ash-St-...,Apartment for sale,null,null,No Data,"$341,000","$1,100/mo","3,482",CA,760 Ash St,2007,"$329,798",$98,96080


In [16]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q1-2020_Multi_Family_Data_page11-15.csv", index=False, header=True)